In [2]:

! pip install python-docx
! pip install spacy_annotator

In [3]:

from docx import Document
from os import walk
from tqdm.notebook import tqdm
import spacy_annotator
import pandas as pd
import spacy

In [20]:

## Do the processing for Jan's dataset

data = pd.read_csv("../file002.csv")
print(len(data))
print(data.head())
# Load model
nlp = spacy.load("en_core_web_sm")


135
   Unnamed: 0                                               Text  \
0           0                                      THIRD SECTION   
1           1                 CASE OF BUCEA AND BUCIA v. ROMANIA   
2           2                         (Application no. 32185/04)   
3           3                                           JUDGMENT   
4           4  This version was rectified on 16 December 2014...   

                                                Word  POS  \
0                               ['THIRD', 'SECTION']  NaN   
1  ['CASE', 'OF', 'BUCEA', 'AND', 'BUCIA', 'v.', ...  NaN   
2   ['(', 'Application', 'no', '.', '32185/04', ')']  NaN   
3                                       ['JUDGMENT']  NaN   
4  ['This', 'version', 'was', 'rectified', 'on', ...  NaN   

                                                 Tag  Word_idx  \
0                             ['B-COURT', 'L-COURT']       NaN   
1  ['O', 'O', 'O', 'O', 'B-CASE', 'I-CASE', 'L-CA...       NaN   
2  ['O', 'O', 'B-APPLI

In [5]:

def annotate_by_sentence2(courts, nlp):
  annotated_data = []

  # For every court
  for court in tqdm(courts):
    annotated_court = []

    #print(court)

    # Remake sentence


    # Get all individual sentences (using spaCy tokenizer)
    sentences = [str(x) for x in nlp(court).sents]

    # For every sentence
    for sentence in sentences:
      annotated_sentence = []
      for token in nlp(sentence):
        annotated_sentence.append((token, token.ent_iob_, token.ent_type_))

      # Add sentence to court list
      annotated_court.append(annotated_sentence)

    # Add court to all data
    annotated_data.append(annotated_court)

  return annotated_data


In [6]:
# Get the proper text
word_data = data['Word']
# Transform into normal text
print(word_data[0])
new_word = []

for sent in word_data:
  new_word.append(sent.replace("['", '').replace("']", "").replace("', '", " "))

print(new_word)

['THIRD', 'SECTION']
['THIRD SECTION', 'CASE OF BUCEA AND BUCIA v. ROMANIA', '( Application no . 32185/04 )', 'JUDGMENT', 'This version was rectified on 16 December 2014 under Rule 81 of the Rules of Court', 'STRASBOURG', '1 July 2014', 'FINAL', '01/10/2014', 'This judgment has become final under Article 44 § 2 of the Convention . It may be subject to editorial revision .', 'In the case of Bucea and Bucia v. Romania ,', 'The European Court of Human Rights ( Third Section ) , sitting as a Chamber compomsed of :', 'Josep Casadevall , President , Alvina Gyulumyan , J n ikuta , Luis L pez Guerra , Johannes Silvis , Valeriu Gri co , Iulia Antoanella Motoc , judges , and Santiago Quesada , Section Registrar ,', 'Having deliberated in private on 10 June 2014 ,', 'Delivers the following judgment , which was adopted on that date :', 'PROCEDURE', 'The case originated in an application ( no . 32185/04 ) against Romania lodged with the Court under Article 34 of the Convention for the Protection of

In [7]:

annotated_data = annotate_by_sentence2(new_word, nlp)

  0%|          | 0/135 [00:00<?, ?it/s]

In [11]:
# SIZE IS OK
print(len(annotated_data))
print(len(data["Word"]))

135
135


In [13]:

# Transform it all into the right format:
# Change format of dataset starting here.
import pandas as pd
import numpy as np

training_sentence = []
sentence_count = 1
training_text = []
training_labels = []
pos = []
for section in annotated_data:
  for entry in section:

    training_sentence.append("Sentence: " + str(sentence_count))
    flag = False

    for datae in entry:
      # text data to clean
      text_data = str(datae[0])
      text_data = text_data.replace("\n", "")
      text_data = text_data.replace("\t", "")

      # Find escape characters to clean
      if "\\" in text_data:
        print(text_data)

      # Update sentence tracking
      if flag:
        training_sentence.append(np.nan)
      else:
        flag = True

      training_text.append(text_data)
      training_labels.append(str(datae[1]) + ' ' + str(datae[2]))
      pos.append("NNS")
      #training_data.append(training_entry)

    #break
    #if sentence_count >= 2:
    #  break
    sentence_count += 1

  #break

print(len(training_sentence))
print(len(training_text))
print(len(training_labels))

5447
5447
5447


In [19]:
df = pd.DataFrame({
  "Sentence #": training_sentence,
  "Word": training_text,
  "POS": pos,
  "Tag": training_labels
                   })

print(df.head())

    Sentence #     Word  POS    Tag
0  Sentence: 1    THIRD  NNS  B ORG
1          NaN  SECTION  NNS  I ORG
2  Sentence: 2     CASE  NNS     O 
3          NaN       OF  NNS     O 
4          NaN    BUCEA  NNS     O 


In [16]:


#print(df.head())

# Reloop to turn back to the other format:

spacy_data = []
paragraph = []
p_word = []
text = ""
p_tag = []
p_text = []
p_pos = []
for index, row in df.iterrows():
  #print(str(row['Sentence #']))
  if str(row['Sentence #']) != "nan":
    if len(p_word) > 0:
      spacy_data.append([text, p_word, p_pos, p_tag])
      text = ""
      p_word = []
      p_tag = []
      p_pos = []

  text += row['Word'] + " "
  p_word.append(row['Word'])
  p_tag.append(row['Tag'])
  p_pos.append('NNS')

  #break

In [18]:
print(spacy_data[0])
spacy_df = pd.DataFrame(spacy_data, columns=["Text", "Word", "POS", "Tag"])
print(len(spacy_df))
print(spacy_df.head())

['THIRD SECTION ', ['THIRD', 'SECTION'], ['NNS', 'NNS'], ['B ORG', 'I ORG']]
231
                                                Text  \
0                                     THIRD SECTION    
1                CASE OF BUCEA AND BUCIA v. ROMANIA    
2                     ( Application no . 32185/04 )    
3                                          JUDGMENT    
4  This version was rectified on 16 December 2014...   

                                                Word  \
0                                   [THIRD, SECTION]   
1         [CASE, OF, BUCEA, AND, BUCIA, v., ROMANIA]   
2               [(, Application, no, ., 32185/04, )]   
3                                         [JUDGMENT]   
4  [This, version, was, rectified, on, 16, Decemb...   

                                                 POS  \
0                                         [NNS, NNS]   
1                [NNS, NNS, NNS, NNS, NNS, NNS, NNS]   
2                     [NNS, NNS, NNS, NNS, NNS, NNS]   
3                    